In [5]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/dianaestrada/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [6]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [7]:
from sklearn import tree
from sklearn.model_selection import train_test_split
import pandas as pd
import os


# Read the CSV and Perform Basic Data Cleaning

In [8]:
df = pd.read_csv("exoplanet_data1.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [9]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition','koi_model_snr','koi_prad','koi_depth','koi_period','koi_impact']]
selected_features.head()


,koi_disposition,koi_model_snr,koi_prad,koi_depth,koi_period,koi_impact
0,CONFIRMED,25.8,2.83,874.8,54.418383,0.586
1,FALSE POSITIVE,76.3,14.60,10829.0,19.899140,0.969
2,FALSE POSITIVE,505.6,33.46,8079.2,1.736952,1.276
3,CONFIRMED,40.9,2.75,603.3,2.525592,0.701
4,CONFIRMED,40.2,2.77,686.0,4.134435,0.762


# Create a Train Test Split

Use `koi_disposition` for the y values

In [10]:
# Assign X (data) and y (target)
X = selected_features.drop("koi_disposition", axis=1)
y = selected_features["koi_disposition"]
print(X.shape, y.shape)


(6991, 5) (6991,)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)


In [12]:
X_train.head()


,koi_model_snr,koi_prad,koi_depth,koi_period,koi_impact
6122,10.8,1.24,123.1,6.768901,0.150
6370,13.8,0.86,114.6,0.733726,0.291
2879,254.3,3.21,641.1,7.652707,0.970
107,38.4,2.25,875.4,7.953547,0.300
29,696.5,12.21,9802.0,4.959319,0.831


In [13]:
y_train.head()


6122         CANDIDATE
6370    FALSE POSITIVE
2879    FALSE POSITIVE
107          CONFIRMED
29           CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [14]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)


In [15]:
# Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Train the Model



In [16]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier


LogisticRegression()

In [17]:
classifier.fit(X_train_scaled, y_train)


LogisticRegression()

In [18]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")


Training Data Score: 0.5300400534045394
Testing Data Score: 0.5194508009153318


# Figure the importance of the Feautures
#### Using a random forest calssifier estimate the importance

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)


0.7191075514874142

In [20]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances


array([0.22527868, 0.20418041, 0.17283412, 0.22383371, 0.17387308])

In [21]:
feature_names = X.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)


[(0.22527867977297472, 'koi_model_snr'),
 (0.22383370930390037, 'koi_period'),
 (0.20418040654957703, 'koi_prad'),
 (0.17387308248281858, 'koi_impact'),
 (0.17283412189072941, 'koi_depth')]

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [22]:
# Create the GridSearchCV model
# Create the SVC Model
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model


SVC(kernel='linear')

In [23]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)


In [24]:
# Train the model with GridSearch
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] ................... C=1, gamma=0.0001, score=0.521, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[CV] ................... C=1, gamma=0.0001, score=0.532, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[CV] ................... C=1, gamma=0.0001, score=0.516, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.522, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.528, to

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [25]:
print(grid.best_params_)
print(grid.best_score_)


{'C': 50, 'gamma': 0.0001}
0.528325413516326


In [26]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)


In [27]:
print('Test Acc: %.3f' % grid.score(X_test_scaled, y_test))


Test Acc: 0.518


In [28]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["FALSE POSITIVE", "CONFIRMED","CANDIDATE"]))


                precision    recall  f1-score   support

FALSE POSITIVE       0.35      0.03      0.05       411
     CONFIRMED       0.40      0.56      0.46       484
     CANDIDATE       0.60      0.73      0.66       853

      accuracy                           0.52      1748
     macro avg       0.45      0.44      0.39      1748
  weighted avg       0.49      0.52      0.46      1748



# Save the Model

In [29]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'model_2.sav'
joblib.dump(model, filename)


['model_2.sav']